In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/more_clean_ver.csv')
data=data.dropna()
dates = data['time'].values
data['time'] = pd.to_datetime(data['time'])
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month
data['day'] = data['time'].dt.day
time = data[['time']]
data = data.drop(columns=['time'])

In [ ]:
features = data[['depth', 'year', 'month', 'day', 'latitude', 'longitude']]
targets = data[['sea_water_temperature', 'sea_water_practical_salinity']]

In [ ]:

features = data[['depth', 'year', 'month', 'day', 'latitude', 'longitude']]
targets = data[['sea_water_temperature', 'sea_water_practical_salinity']]


X_train, X_test, y_train, y_test, time_train, time_test = train_test_split(
    features, targets, time, test_size=0.2, random_state=42
)


print(y_test.head())
print(time_test.head())


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)


y_pred = model.predict(X_test_scaled)


y_test_df = pd.DataFrame(y_test.values, columns=['actual_temperature', 'actual_salinity'])
time_test_df = pd.DataFrame(time_test.values, columns=['date'])


print(y_test_df.head())
print(time_test_df.head())


preds_df = pd.DataFrame(y_pred, columns=['predicted_temperature', 'predicted_salinity'])


results_df = pd.concat([time_test_df.reset_index(drop=True),
                        y_test_df.reset_index(drop=True),
                        preds_df.reset_index(drop=True)], axis=1)




          sea_water_temperature  sea_water_practical_salinity
11432289              12.226200                       29.3180
3389503                6.255499                       33.9520
6876498                8.670700                       32.4024
9639155                9.134200                       31.3349
12124092               6.513000                       33.1564
                              time
11432289 2019-07-05 07:01:23+00:00
3389503  1992-08-02 04:00:00+00:00
6876498  2004-06-23 05:24:49+00:00
9639155  2013-11-06 20:06:08+00:00
12124092 2021-06-07 07:54:48+00:00


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


In [ ]:
results_csv_path = 'actual_vs_predicted.csv'
results_df.to_csv(results_csv_path, index=False)

print("CSV file exported.")